In [62]:
# pip install tf-keras

In [1]:
import cv2
import torch
import numpy as np
from PIL import Image
import tempfile
from deepface import DeepFace
import torch.nn as nn
from transformers import pipeline, AutoImageProcessor, AutoModelForImageClassification

In [2]:
image_path = 'c:/Users/dalex/Downloads/sad.jpg'
image = cv2.imread(image_path)

original_path= ''
modified_path = ''

with tempfile.NamedTemporaryFile(suffix='.jpg', delete=False) as temp_file:
    temp_file_path = temp_file.name
    cv2.imwrite(temp_file_path, image)
    original_path = temp_file_path

image_bgr = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
with tempfile.NamedTemporaryFile(suffix='.jpg', delete=False) as temp_file:
    temp_file_path = temp_file.name
    cv2.imwrite(temp_file_path, image_bgr)
    modified_path = temp_file_path
    
print('Original image saved to', original_path)
print('Modified image saved to', modified_path)

In [32]:
no_face_path = 'c:/Users/dalex/AppData/local/temp/tmpx92yanqz.jpg'
try:
    objs = DeepFace.analyze(img_path=original_path, actions=['emotion'])
    results = objs[0]['emotion']
    emotions = {k: v for k, v in sorted(results.items(), key=lambda x: x[1], reverse=True)}
except ValueError as e:
    emotions = {'angry': 0.0,
                'neutral': 0.0,
                'sad': 0.0,
                'happy': 0.0,
                'fear': 0.0,
                'surprise': 0.0,
                'disgust': 0.0
                }
    
emotions

In [17]:

if detections is not None and len(detections) > 0:
    print("Face detected!")
else:
    print("No face detected.")

In [13]:
original_objs = DeepFace.analyze(img_path = original_path, actions = ['detect_face', 'emotion'])

In [11]:
emotions = original_objs[0]['emotion']
emotions

In [12]:
sorted_values = {k: v for k, v in sorted(emotions.items(), key=lambda x: x[1], reverse=True)}
print('Sorted values:', sorted_values)

Hugginface model With pipeline

In [53]:
task = "image-classification"
model_id = "dima806/facial_emotions_image_detection"
pipe = pipeline(task, model_id, top_k=7)

def predict(image_path: str) -> dict[str, float]:
    result = pipe(image_path)

    output = {}
    for entry in result:
        output[entry['label']] = entry['score'] * 100

    sorted_values = {k: v for k, v in sorted(output.items(), key=lambda x: x[1], reverse=True)}

    return sorted_values

result_1 = predict(image_path)
result_1

Without pipeline

In [54]:
processor = AutoImageProcessor.from_pretrained("dima806/facial_emotions_image_detection", output_attentions=True)
model = AutoModelForImageClassification.from_pretrained("dima806/facial_emotions_image_detection", output_attentions=True)
assert model.config.output_attentions == True

In [75]:
image = cv2.imread(image_path)
cv2.imshow('Frame', image)
cv2.waitKey(0)


image_bgr = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
cv2.imshow('Frame', image_bgr)
cv2.waitKey(0)
cv2.destroyAllWindows()
# image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
# pil_image_1 = Image.fromarray(image_rgb)
# pil_image_2 = Image.fromarray(image)

In [46]:
inputs = processor(image, return_tensors="pt")
outputs = model(**inputs)
logits = outputs.logits
attention_weights = outputs.attentions
print(logits)
print(attention_weights)

In [22]:
logits

In [67]:

m = nn.Softmax(dim=0)
values = m(logits[0])

w_values = np.zeros(7)
for i in range(7):
    result = values[i].item() * attention_weights[i]
    w_values[i] = torch.sum(result)

# Convert to percentage
total = np.sum(w_values)
w_values = w_values*100/total

values_dict = dict(zip(model.config.id2label.values(), w_values))
result_2 = {k: v for k, v in sorted(values_dict.items(), key=lambda x: x[1], reverse=True)}

In [68]:
print(result_1)
print(result_2)